# Lesson 3: Agentic Search

## High-level overview


![image](./l3-1.png)

Let's understand how an agent might use it. In a zero shot learning an agent would receive a prompt and will produce an answer based on its static weights of the model.
As powerful as it's proven to be, there are many limitations to this process.
First, the data on this is dynamic.
So we couldn't, for instance, ask about the scores from the game last night.
Secondly, in many use cases, we would want to know the sources of the information provided in the result.
This can reduce hallucinations and smooth the friction of this human-computer interaction.

## Low-level overview

![image](./l3-2.png)

For example, if an agent would ask "How is the weather in San Francisco?" The search tool should use the weather APl for best results. the job doesn't end with finding the correct source.
The search tool would then have to extract only the relevant information to the subquery.

A basic implementation of this can be achieved through a process of chunking the source, and run a quick vector search to retrieve the top-K chunks. After retrieving the data from its source, the search tool would then score the results and filter out the less relevant information.

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?", include_answer=True)

# print the answer
result["answer"]

"The new Nvidia Blackwell GPU is built upon a 208 billion transistor, dual-die architecture with a 10 TB/s chip-to-chip interconnect. It is designed specifically for datacenter-scale generative AI applications and is named after David Harold Blackwell, a notable statistician and mathematician. The Blackwell GPU offers a 30X performance increase and a 25X decrease in total cost of ownership (TCO) over its predecessor, the H100. Additionally, multiple Blackwell GPUs can be interconnected using Nvidia's Quantum-X800 and Spectrum-X800 Ethernet at speeds up to 800 GB/s."

## Regular search

In [3]:
# choose location (try to change to your own city!)

city = "London"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [4]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

https://weather.com/en-GB/weather/hourbyhour/l/London+England?canonicalCityId=805c6df1c2518951d4dffa28cf6e483358644c3131109dff6c9a8a2c265fbc67
https://weather.com/en-GB/weather/today/l/7517a52d4d1815e639ae1001edb8c5fda2264ea579095b0f28f55c059599e074
https://weather.com/weather/hourbyhour/l/London+England+United+Kingdom?canonicalCityId=805c6df1c2518951d4dffa28cf6e483358644c3131109dff6c9a8a2c265fbc67
https://weather.com/weather/today/l/ae8230efd4bc57fdf721a02c7eb2b88c56aa6e71d73666328e33af3ea2039032132e24ae91b6a07862c5091a9d95a4b8
https://weather.com/weather/tenday/l/London+England+United+Kingdom?canonicalCityId=805c6df1c2518951d4dffa28cf6e483358644c3131109dff6c9a8a2c265fbc67
https://weather.com/en-GB/weather/tenday/l/7bc0022212d6873b8226ac81bad9714bc956aef8851519b045d7750eb3ea4745


In [5]:
# Sai: We don't want to get the link, but instead, we need to get the content from the links
# so we have below helper function to get the content
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [6]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://weather.com/en-GB/weather/hourbyhour/l/London+England?canonicalCityId=805c6df1c2518951d4dffa28cf6e483358644c3131109dff6c9a8a2c265fbc67


<body><div class="appWrapper DaybreakLargeScreen LargeScreen lightTheme twcTheme DaybreakLargeScreen--appWrapper--3kKUE gradients--cloudyFoggyDay--R2hTM gradients--cloudyFoggyDay-top--3dKG9" id="appWrapper"><div class="region-meta"><div class="removeIfEmpty" id="WxuHtmlHead-meta-"></div></div><div class="region-topAds regionTopAds DaybreakLargeScreen--regionTopAds--2kcLJ"><div class="removeIfEmpty" id="WxuAd-topAds-53dce052-5465-4609-a555-c3a20ab64ab0"><div class="adWrapper BaseAd--adWrapper--1dWzA BaseAd--card--2G19u BaseAd--hide--3LkRr"><div class="adLabel BaseAd--adLabel--1LMu9">Advertisement</div><div class="ad_module BaseAd--ad_module--DeGCt subs-undefined BaseAd--placeholder--3IQan" id="WX_Hidden"></div></div></div><div class="removeIfEmpty" id="WxuAd-topAds-fe926b10-58bc-448a-ab09-47e692334250"><div class="adWrapper BaseAd--adW

In [7]:
# extract the weather_data text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/en-GB/weather/hourbyhour/l/London+England?canonicalCityId=805c6df1c2518951d4dffa28cf6e483358644c3131109dff6c9a8a2c265fbc67


recents Special Forecasts Hourly Weather - London, England Moderate rain warning Wednesday, 4 September 15:00 Mostly Cloudy 16:00 Partly Cloudy 17:00 Mostly Cloudy 18:00 Mostly Cloudy 19:00 Mostly Cloudy 20:00 Cloudy 21:00 Cloudy 22:00 Cloudy 23:00 Cloudy Thursday, 5 September 0:00 Cloudy 1:00 Mostly Cloudy 2:00 Cloudy 3:00 Cloudy 4:00 Cloudy 5:00 Cloudy 6:00 Cloudy 7:00 Cloudy 8:00 Few Showers 9:00 Showers 10:00 Showers 11:00 Showers 12:00 Showers 13:00 Showers 14:00 Rain 15:00 Showers 16:00 Cloudy 17:00 Cloudy 18:00 Cloudy 19:00 Cloudy 20:00 Cloudy 21:00 Cloudy 22:00 Cloudy 23:00 Cloudy Friday, 6 September 0:00 Cloudy 1:00 Showers 2:00 Showers 3:00 Mostly Cloudy 4:00 Partly Cloudy 5:00 Partly Cloudy 6:00 Mostly Clear 7:00 Mostly Sunny 8:00 Showers 9:00 Partly Cloudy 10:00 Mostly Cloudy 11:00 Few Showers 12:00 Showers 13:00 Few Showe

## Agentic Search with Tavily

In [8]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'London', 'region': 'City of London, Greater London', 'country': 'United Kingdom', 'lat': 51.52, 'lon': -0.11, 'tz_id': 'Europe/London', 'localtime_epoch': 1725458496, 'localtime': '2024-09-04 15:01'}, 'current': {'last_updated_epoch': 1725458400, 'last_updated': '2024-09-04 15:00', 'temp_c': 19.0, 'temp_f': 66.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 360, 'wind_dir': 'N', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.09, 'precip_in': 0.0, 'humidity': 52, 'cloud': 75, 'feelslike_c': 19.0, 'feelslike_f': 66.2, 'windchill_c': 19.2, 'windchill_f': 66.6, 'heatindex_c': 19.2, 'heatindex_f': 66.6, 'dewpoint_c': 10.2, 'dewpoint_f': 50.3, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 4.0, 'gust_mph': 10.5, 'gust_kph': 16.8}}


In [9]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "London",
        "region": "City of London, Greater London",
        "country": "United Kingdom",
        "lat": 51.52,
        "lon": -0.11,
        "tz_id": "Europe/London",
        "localtime_epoch": 1725458496,
        "localtime": "2024-09-04 15:01"
    },
    "current": {
        "last_updated_epoch": 1725458400,
        "last_updated": "2024-09-04 15:00",
        "temp_c": 19.0,
        "temp_f": 66.2,
        "is_day": 1,
        "condition": {
            "text": "Partly cloudy",
            "icon": "//cdn.weatherapi.com/weather/64x64/day/116.png",
            "code": 1003
        },
        "wind_mph": 5.6,
        "wind_kph": 9.0,
        "wind_degree": 360,
        "wind_dir": "N",
        "pressure_mb": 1017.0,
        "pressure_in": 30.03,
        "precip_mm": 0.09,
        "precip_in": 0.0,
        "humidity": 52,
        "cloud": 75,
        "feelslike_c": 19.0,
        "feelslike_f": 66.2,
        "windchill_c": 19.2,
        "windc

As you can see, this is not the answer I would want to see as a human. But this is the exact answer an agent would want, a structured data.
